# Deploying AutoML model as web service to Azure Container Instance (ACI)

This notebook shows the steps for deploying a service: registering a model, creating an image, provisioning a cluster (one time action), and deploying a service to it. We then test and delete the service, image and model.


In [1]:
import os
import requests


import azureml.core
from azureml.core import Experiment, Workspace
from azureml.core.image import ContainerImage
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute import ComputeTarget
from azureml.exceptions import ProjectSystemException, UserErrorException

import sys
sys.path.append("../../")

In [2]:
# Check core SDK version number
print(f"Azure ML SDK Version: {azureml.core.VERSION}")

Azure ML SDK Version: 1.0.43


## Get workspace

Load existing workspace from the config file info. 
The config file can be downloaded from the portal and should be placed in the same folder as this notebook.

In [3]:
# Let's load the workspace from the configuration file
ws = Workspace.from_config()
print("Workspace was loaded successfully from the configuration file")

Workspace was loaded successfully from the configuration file


In [4]:
# Print the workspace attributes
print(f'Workspace name: {ws.name}\n \
      Azure region: {ws.location}\n \
      Subscription id: {ws.subscription_id}\n \
      Resource group: {ws.resource_group}')

Workspace name: MAIDAPNLP
       Azure region: eastus2
       Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
       Resource group: nlprg


## Create a new workspace
Create new workspace if you don't have one already. 

In [ ]:
# Let's define these variables here - These pieces of information can be found on the portal
subscription_id = os.getenv("SUBSCRIPTION_ID", default="<our_subscription_id>")
resource_group =  os.getenv("RESOURCE_GROUP", default="<our_resource_group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<our_workspace_name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="<our_workspace_region>")


# load one directly from Azure, if it already exists (exist_ok=True).
# If it does not exist, let's create a workspace from scratch
ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=workspace_region,
                      create_resource_group=True,
                      exist_ok=True
                      )
ws.write_config()
print("Workspace was loaded successfully from Azure")

In [ ]:
# Print the workspace attributes
print(f'Workspace name: {ws.name}\n \
      Azure region: {ws.location}\n \
      Subscription id: {ws.subscription_id}\n \
      Resource group: {ws.resource_group}')

In [ ]:
service = Webservice(workspace=ws, name='aci-service-1')
print(service.scoring_uri)

## Register the model 
Register an existing trained model, add descirption and tags.

In [5]:
model = Model.register(
    model_path = "sentence_similarity_regressor.pkl",
    model_name = "automl-ss",
    tags = {"Model": "automl-regressor"},
    description = "using google universal encoder model",
    workspace = ws
)

Registering model automl-ss


In [ ]:
os.getcwd()

## Create an image
Create an image using the registered model the script that will load and run the model.
score.py for infersent encoder

In [14]:
%%writefile score.py
import numpy as np
import time
import os
import json
import pickle
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model


def init():
    global model
    model_path = Model.get_model_path('automl-ss') # this should get the model path
    model = joblib.load(model_path)
    
    
def run(rawdata):
    try:
        data = json.loads(rawdata)['data']
        data = np.array(data)
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result":result.tolist()})


Overwriting score.py


Create an env.yaml using conda_dependencies from azureml.core package. Include all pip or conda depedencies for the model inage here

In [ ]:
'''
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','pytorch','torchvision'], python_version = '3.6.8')

myenv.add_channel('conda-forge')

with open("automlenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
'''
    
    

In [20]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn','py-xgboost<=0.80'],
                                 pip_packages=['azureml-sdk[automl]'], python_version = '3.6.8')

conda_env_file_name = 'automlenv.yml'
myenv.save_to_file('.', conda_env_file_name)

'automlenv.yml'

In [13]:
with open(conda_env_file_name, 'r') as cefr:
    content = cefr.read()

with open(conda_env_file_name, 'w') as cefw:
    cefw.write(content.replace(azureml.core.VERSION, dependencies['azureml-sdk']))

NameError: name 'dependencies' is not defined

### Image creation

In [21]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "automlenv.yml",
                                                  description = "Image with automl model",
                                                  tags = {'area': "nlp", 'type': "sentencesimilarity automl"})

image = ContainerImage.create(name = "automl-image",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)


Creating image
Running.
NotStarted..............................................
Succeeded
Image creation operation finished for image automl-image:5, operation "Succeeded"


If the above step fails then use below command to see logs 

In [ ]:

print(image.image_build_log_uri) 

## Deploy image as web service to Azure Container Instance

In [22]:
#Set the web service configuration (using default here)
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1)

In [23]:
# deploy image as web service
aci_service_name ='aci-service-automl-4'
aci_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aci_service_name,
                                           image = image,
                                           deployment_config = aci_config)

aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

Creating service
Running......................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


Fetch logs to debug incase of failures. 

In [19]:
print(aci_service.get_logs())

2019-06-14T17:23:50,516485899+00:00 - rsyslog/run 
2019-06-14T17:23:50,519038316+00:00 - nginx/run 
2019-06-14T17:23:50,518157176+00:00 - iot-server/run 
2019-06-14T17:23:50,524561171+00:00 - gunicorn/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-06-14T17:23:50,635289566+00:00 - iot-server/finish 1 0
2019-06-14T17:23:50,637371362+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.6.0
Listening at: http://127.0.0.1:9090 (13)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 45
'pattern' package not found; tag filters are not available for English
Initializing logger
Starting up app insights client
Starting up request id generator
Starting up app insight hooks
Invoking user's init function
2019-06-14 17:23:57,403 | azureml.core.run | DEBUG | Could not load run context Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run., 

In [ ]:
# get already deployed service 
Webservice.list(workspace = ws)

## Test the web service using run method 
We test the web sevice by passing data. Run() method retrieves API keys behind the scenes to make sure that call is authenticated. The run method expects a valid json format. 

In [ ]:
import json
sentences = ["This is sentence 1", "This is sentence 2"]
#sentences = ['Everyone really likes the newest benefits', 'The Government Executive articles housed on the website are not able to be searched .', 'I like him for the most part , but would still enjoy seeing someone beat him .', 'My favorite restaurants are always at least a hundred miles away from my house .', 'I know exactly .', 'We have plenty of space in the landfill .', 'I did that all through college but it never worked', "Most of Mrinal Sen 's work can be found in European collections .", 'THe strike price could be $ 8 .', 'Would you rise up and defeaat all evil lords in the town ?', 'Go downwards to one of the gates , all of which will lead you into the cathedral .', "The Tamils ' bhakti movement froze the previously warm ritual of Hinduism .", 'Everyone involved was the same age .', 'Severn said the people were not welcome there .', "We do n't loan a lot of money .", "I do n't know how cold it got last night .", 'I need a way to add something extra .', 'Sometimes it is amusing to see what the hologram creates .', 'Gross national saving was highest this year .', 'The amount of lost mail is huge and really impacts mail volume', "You do n't want to push the button lightly , but rather punch it hard .", 'It has a buffet .', 'Clinton said that Monica Lewinsky made unwanted sexual advances during her time as a journalist in the White House .', 'Leading organizations want to be sure their employees are safe .', "No , they would n't go there .", 'This is how things are and there are no apologies about it .', 'Pope John Paul II also visited in 1983 .', 'Tourism is not very big in Spain .', "She had changed a lot since the last time we 'd seen her .", 'I can not wait for it to happen .', 'The 1931 Malay census was an alarm bell .', 'Many youth are lazy .', 'Interest rates should increase to increase saving .', 'There is a British publication called the Sun .', 'He loved how peaceful the village was .', 'The slopes between the Vosges and Rhine Valley are the only place appropriate for vineyards .', 'These deviations mostly involve failure to apply software updates in a timely manner .', 'Joan of Arc sacrificed her life at Rouen , which became an enduring symbol of opposition to tyranny .', "Senior Executive 's have been studies on various aspects to reach the expectations .", "It 's impossible to have a plate hand-painted to your own design in Hong Kong .", 'Jewelry and duty-free shops are an interesting place to buy goods .', 'Jobs never held onto an idea for long .', "Democracies probably wo n't go to war unless someone attacks them on their soil", 'There is no information on the mentality of the man , extraordinary or not , contained within the thin-paged book .', "The military did n't pay for her education .", 'These men filed their midterm exams from home .', 'Not many couples with kids can save up for retirement .', "They dodged the draft , I 'll have you know .", 'Lincoln took his hat off .', 'The stock market can experience much worse damage .', "Missouri was happy to continue it 's planning efforts .", 'He had recently seen pictures depicting those things .', 'See , there is a well known hero here .', 'At the end of the fourth century was when baked goods flourished .', 'No one plays sports on the weekend .', 'Harlem did a great job', 'The cushion will likely be spent in under two years .', 'I am beyond proud .', 'She has chose to live a hollow life .', 'All invasions of privacy should be severely punished , because it will teach the criminals that it is not worth doing .', 'The medicine he had taken had worked well .', 'New people chose to donate to the cause', 'Business intelligence industry is a new and promising field of study .', 'The gross cost .', 'Jim Lehrer has no credibility whatsoever .', 'Tuppence floated into the air .', 'Jamus looked over the mare .', "There 's nothing worth seeing in the tourist offices .", 'Changes were made to the Grant Renewal Application to provide extra information to the LSC .', 'This building job will be very difficult to complete .', 'Poirot did not look at me .', 'I want you to follow him , so watch for the signal that I give .', "Cook 's American Business Alliance caused shares of stock to come back .", 'I was dreadfully worried about many things .', 'I yelled at the top of my lungs .', "Women 's bodies belong to themselves , they should decide what to do with it .", 'You can sub it even if you do not want to get your hands on it .', 'What was unique ?', 'I know because I learned it growing up', 'Whole life policies are a type of life insurance that only cover the insured person until retirement from the workforce .', "Justice Kennedy does n't care if the Supreme Court Reporters from 1790 to 1998 are thrown away .", 'The whole countryside is scattered with small villages .']

data = {'data': sentences}
data = json.dumps(data)
print(type(data))
print(data)

In [30]:
# load multiple sentences
import pandas as pd
import json 

sentences = []
data = pd.read_csv("testing_set.csv")
train_y = data['score'].values.flatten()
train_x = data.drop("score", axis=1).values

print(type(train_x))

train_x = train_x.tolist()
data = {'data': train_x}
data = json.dumps(data)
print(len(data))

#print(data)

<class 'numpy.ndarray'>
21256649


In [31]:

#embeddings = aci_service.run(input_data = data)

score = aci_service.run(input_data = data)

# embeddings will print the error message incase error occurs.
print('nb sentences encoded : {0}'.format(len(score)))
print(score)

nb sentences encoded : 27005
{"result": [2.214819090898734, 3.7706938042747375, 3.0781882683909005, 3.869533669921049, 1.2883272585204812, 1.7202466111769048, 3.4733980914072746, 2.0485645854056975, 2.631252821328933, 1.7006235165222483, 1.7006235165222483, 4.054679101385872, 1.4535831498906595, 3.353173122204415, 2.3015902963884756, 1.6734629175557592, 4.201997453073964, 3.400864121559613, 3.301564203308278, 1.6538660845723256, 2.35315904417188, 1.609571059570872, 3.876787615570435, 3.8296621730786713, 1.440571297828224, 3.2825800116509543, 1.2502532902291936, 2.9001951247264017, 1.9283168530468258, 3.2201117295648785, 3.63135940230565, 3.4339657240569466, 2.155529585768872, 3.9452969467799903, 2.5094914855986294, 0.17263563772765547, 1.282440235369095, 2.827545602576309, 3.0652856532839876, 0.7765182761102716, 3.0552306261904016, 3.022741045259836, 3.4709387163405805, 1.0764230879073133, 2.4546017874682513, 1.4162360202720037, 1.0173504130231579, 0.7038859191380052, 3.120444383082996

In [40]:
from scipy.stats import pearsonr
#print(train_y)
result = json.loads(score)
output = result["result"]
print(pearsonr(output, train_y)[0])

0.7817562992742639


Fetch logs to debug incase of failures. 

In [ ]:
print(aci_service.get_logs())

## Clean up
Delete the service, image and model.

In [ ]:
#aci_service.delete()
#image.delete()
#model.delete()